In [1]:
import tensorflow as tf
from tensorflow import keras
from keras import layers
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

In [2]:
from keras.datasets import mnist

(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

11501568/11490434 [==============================] - 0s 0us/step


In [3]:
train_images.shape

(60000, 28, 28)

In [4]:
x_train = train_images.reshape((train_images.shape[0], 28 * 28))
x_train = x_train.astype("float32") / 255

x_test = test_images.reshape((test_images.shape[0], 28 * 28))
x_test = x_test.astype("float32") / 255

y_train = train_labels.astype("float32")
y_test = test_labels.astype("float32")

In [5]:
model = keras.Sequential()
model.add(keras.layers.Dense(128, activation="relu"))
model.add(keras.layers.Dense(10, activation="softmax"))

### fit() method implementation

In [6]:
loss_fn = keras.losses.SparseCategoricalCrossentropy()
optimizer = keras.optimizers.RMSprop()
metrics = [keras.metrics.SparseCategoricalAccuracy()]
loss_metric = keras.metrics.Mean()

def training_step(inputs, targets):
    with tf.GradientTape() as tape:
        predictions = model(inputs, training=True)
        loss = loss_fn(targets, predictions)
    gradients = tape.gradient(loss, model.trainable_weights)
    optimizer.apply_gradients(zip(gradients, model.trainable_weights))
    logs = {}
    for metric in metrics:
        metric.update_state(targets, predictions)
        logs[metric.name] = metric.result()
    loss_metric.update_state(loss)
    logs["loss"] = loss_metric.result()
    return logs

def reset_metrics():
    for metric in metrics:
        metric.reset_state()
    loss_metric.reset_state()

def fit(x_train, y_train, batch_size, epochs):
    training_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))
    training_dataset = training_dataset.batch(batch_size)
    for epoch in range(epochs):
        reset_metrics()
        for inputs_batch, labels_batch in training_dataset:
            logs = training_step(inputs_batch, labels_batch)
        print(f"Epoch {epoch + 1} --> ", end="")
        for key, value in logs.items():
            print(f"{key} : {value:.4f}  ", end="")
        print("")

In [7]:
fit(model, x_train, y_train, 128, 10)

Epoch 1 --> sparse_categorical_accuracy : 0.9038  loss : 0.3517  
Epoch 2 --> sparse_categorical_accuracy : 0.9504  loss : 0.1755  
Epoch 3 --> sparse_categorical_accuracy : 0.9645  loss : 0.1244  
Epoch 4 --> sparse_categorical_accuracy : 0.9728  loss : 0.0957  
Epoch 5 --> sparse_categorical_accuracy : 0.9779  loss : 0.0772  
Epoch 6 --> sparse_categorical_accuracy : 0.9818  loss : 0.0640  
Epoch 7 --> sparse_categorical_accuracy : 0.9851  loss : 0.0540  
Epoch 8 --> sparse_categorical_accuracy : 0.9875  loss : 0.0463  
Epoch 9 --> sparse_categorical_accuracy : 0.9896  loss : 0.0400  
Epoch 10 --> sparse_categorical_accuracy : 0.9910  loss : 0.0348  


### evaluate() method implementation

In [10]:
def test_step(inputs, targets):
    predictions = model(inputs, training=False)
    loss = loss_fn(targets, predictions)
    logs = {}
    for metric in metrics:
        metric.update_state(targets, predictions)
        logs["val_" + metric.name] = metric.result()
        loss_metric.update_state(loss)
        logs["val_loss"] = loss_metric.result()
        return logs

def evaluate(x_test, y_test, batch_size):
    val_dataset = tf.data.Dataset.from_tensor_slices((x_test, y_test))
    val_dataset = val_dataset.batch(batch_size)
    reset_metrics()
    for inputs_batch, targets_batch in val_dataset:
        logs = test_step(inputs_batch, targets_batch)
    print("Evaluation --> ", end="")
    for key, value in logs.items():
        print(f"{key} : {value:.4f}  ", end="")
    print("")

In [11]:
evaluate(x_test, y_test, 128)

Evaluation --> val_sparse_categorical_accuracy : 0.9736  val_loss : 0.0931  
